In [0]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



In [0]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
get_ipython().system('pip3 install kfp --upgrade')


 # Kubeflow Pipelines container building sample

 Reference documentation:
 * https://www.kubeflow.org/docs/pipelines/sdk/build-component/
 * https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/

 ## Setup

In [0]:
experiment_name = 'container_building'


 ## Create a python function

In [0]:
def add(a: float, b: float) -> float:
 '''Calculates sum of two arguments'''
 
 print("Adding two values %s and %s" %(a, b))
 
 return a + b


 ## Build a new container for and use is as the base image for components.
 Build and push a new container image that includes some required packages.
 Use the new container image as the base image for components.
 Create a new component from the above function.
 The return value "add_op" represents a step that can be used directly in a pipeline function.

In [0]:
import kfp
from kfp.containers import build_image_from_working_dir
from kfp.components import func_to_container_op

from pathlib import Path
import tempfile

packages = ['google-api-python-client == 1.7.0']
# Setting up a directory that contains requirements.txt file with our required packages
with tempfile.TemporaryDirectory() as working_dir: # By default build_image_from_working_dir just uses the current directory
 (Path(working_dir) / 'requirements.txt').write_text('\n'.join(packages))

# Building and pushing new container image
image_with_packages = build_image_from_working_dir(
 working_dir=working_dir, # Optional. Default is the current directory
 base_image='google/cloud-sdk:latest', # Optional
 #image_name=TARGET_IMAGE, # Optional. Default is gcr.io/<project_id>/<notebook_id>/kfp_container
)

add_op = func_to_container_op(add, base_image=image_with_packages)



 ### Build a pipeline using this component

In [0]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Calculation pipeline',
   description='A sample pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a='1',
   b='7',
   c='17',
):
 #Passing pipeline parameter and a constant value as operation arguments
 add_task = add_op(a, b) #Returns a dsl.ContainerOp class instance. 
 
 #You can create explicit dependancy between the tasks using xyz_task.after(abc_task)
 add_2_task = add_op(b, c)
 
 add_3_task = add_op(add_task.output, add_2_task.output)


 ## Submit the pipeline for execution

In [0]:
arguments = {'a': '7', 'b': '8'}
kfp.Client().create_run_from_pipeline_func(pipeline_func=calc_pipeline, arguments=arguments, experiment_name=experiment_name)

# This should output link that leads to the run information page. 
# Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working
